In [3]:
import random
def get_n_states_at_random(n,numStates):
    states_list = [] 
    for i in range(n):
        rand_state = random.randint(0,numStates)
        while rand_state in states_list: 
            rand_state = random.randint(0,numStates)
        states_list.append(rand_state)
    return states_list 

In [4]:
import os
def create_dir(dirName):
    # Create target directory & all intermediate directories if don't exists
    if not os.path.exists(dirName):
        os.makedirs(dirName)
        print("Directory " , dirName ,  " Created ")
    else:    
        print("Directory " , dirName ,  " already exists")    
        

In [5]:
def get_labels(prism_model_lines,state_var_name, label_var_name):
    labels={}
    text_to_find = "label"
    text_to_scrap = label_var_name
    text_to_scrap2 = state_var_name
    for i in range(len(prism_model_lines)):
        line = prism_model_lines[i]
        if text_to_find in line:
            if text_to_scrap in line and text_to_scrap2 in line:
                wp_name = line[line.find(text):line.rfind('"')]
                #print wp_name 
                wp_num = line[line.find("(")+1:line.find(")")]
                #print wp_num 
                wp_name = wp_name[len(text_to_scrap):]
                wp_num = wp_num[len(text_to_scrap2+"="):]
                #print wp_name 
                #print wp_num
                labels[int(wp_name)]=int(wp_num)
    return labels

In [6]:
def get_lines(prism_model_lines,startword,endword):
    start = -1
    end = -1
    for i in range(len(prism_model_lines)):
        line = prism_model_lines[i]
        if endword in line: 
            end = i
        elif startword in line:
            start = i 
        if (start != -1 and end != -1):
            break 
    return (start+1,end,prism_model_lines[start+1:end])    

In [7]:
def get_module_lines(prism_model_lines):
    return get_lines(prism_model_lines,"module","endmodule")

In [8]:
def get_reward_lines(prism_model_lines):
    return get_lines(prism_model_lines,"rewards","endrewards")

In [9]:
def add_failure_to_states(state, module_lines):
    print "Adding failstates to "+str(state)
    state_line = "1.0:(waypoint'="+str(state)+")"
    rep_line = "p:(waypoint'="+str(state)+") + (1-p):(waypoint'=failstate)"
    new_module_lines = []
    for line in module_lines:
        if state_line in line:
            line = line.replace(state_line,rep_line)
        new_module_lines.append(line)
    return new_module_lines

In [10]:
def add_door_to_oneside(doornum,state1,state2,module_lines):
    print "Adding door"+str(doornum)+" to " +str(state1) +" and "+str(state2)
    #add doors to state 1 
    #so look for waypoint=state1 
    state_line = "(waypoint="+str(state1)+")"
    rep_line = "(waypoint="+str(state1)+") & (door"+str(doornum)+"=open)"
    state_line2 = "waypoint'="+str(state2)
    new_module_lines = [] 
    for line in module_lines:
        if state_line in line:
            print line
            if state_line2 in line: 
                print line
                line = line.replace(state_line,rep_line) 
        new_module_lines.append(line)
    return new_module_lines

In [11]:
def add_door_check(doornum,state1,state2,module_lines,reward_lines):
    door_check_action = "checkdoor"+str(doornum)
    #'Hey %(name)s, there is a 0x%(errno)x error!' % {
    #"name": name, "errno": errno }
    print state1 
    print state2
    line_to_add="[checkdoor%(doorno)d] (waypoint=%(state)d) & (door%(doorno)d=unknown) -> p:(door%(doorno)d'=open) + (1-p):(door%(doorno)d'=closed);\n" % {"doorno":doornum,"state":state1}
    print line_to_add
    module_lines.append(line_to_add)
    line_to_add="[checkdoor%(doorno)d] (waypoint=%(state)d) & (door%(doorno)d=unknown) -> p:(door%(doorno)d'=open) + (1-p):(door%(doorno)d'=closed);\n" % {"doorno":doornum,"state":state2}
    print line_to_add
    module_lines.append(line_to_add)
    line_to_add="[checkdoor%(doorno)d] (waypoint=%(state)d) & (door%(doorno)d=unknown):1.0;\n" % {"doorno":doornum,"state":state1}
    print line_to_add
    reward_lines.append(line_to_add)
    line_to_add="[checkdoor%(doorno)d] (waypoint=%(state)d) & (door%(doorno)d=unknown):1.0;\n" % {"doorno":doornum,"state":state2}
    print line_to_add
    reward_lines.append(line_to_add)
    return (module_lines,reward_lines)

In [12]:
#doornum=1
#state1 = 2
#line_to_add="[checkdoor%(doorno)x] (waypoint=%(state)x) & (door%(doorno)x=unknown) -> p:(door%(doorno)x'=open) + (1-p):(door%(doorno)x'=closed);" % {"doorno":doornum,"state":state1}
#print line_to_add

In [13]:
def add_door_to_bothsides(doornum,state1,state2,module_lines,reward_lines):
    module_lines = add_door_to_oneside(doornum,state1,state2,module_lines)
    module_lines = add_door_to_oneside(doornum,state2,state1,module_lines)
    print state1 
    print state2
    return add_door_check(doornum,state1,state2,module_lines,reward_lines)

In [14]:
def create_new_prism_model_lines(new_module_lines,module_start,module_end,prism_model_lines):
    new_prism_model = list(prism_model_lines)
    new_prism_model[module_start:module_end]=new_module_lines
    return new_prism_model

In [15]:
def create_new_prism_model_lines(new_module_lines,module_start,module_end,new_reward_lines,reward_start,reward_end,prism_model_lines):
    temp_new_prism_model = list(prism_model_lines)
    new_prism_model = temp_new_prism_model[0:module_start]
    for i in range(len(new_module_lines)):
        new_prism_model.append(new_module_lines[i])
    for i in range(module_end,reward_start):
        new_prism_model.append(temp_new_prism_model[i])
    #new_prism_model[module_start:module_end]=new_module_lines
    for i in range(len(new_reward_lines)):
        new_prism_model.append(new_reward_lines[i])
    #new_prism_model[reward_start:reward_end]=new_reward_lines
    for i in range(reward_end,len(temp_new_prism_model)):
        new_prism_model.append(temp_new_prism_model[i])
    return new_prism_model

In [16]:
def write_model_to_file(filename,model_lines):
    f=open(filename,"w")
    f.writelines(model_lines)
    f.close()

def create_model_name(filename,suffix,model_ext):
    nfilename = filename+"_"+suffix+model_ext#filename+"_fs"+str(num_fail_states)+model_ext   
    return nfilename

In [17]:
def get_init_state_for_model_num(init_states,num_model):
    #init_states = [21,24,6,15,5,1,19,8]
    return init_states[num_model]

In [18]:
def modify_init_line(module_lines,num_model,init_states):
    init_line_text="waypoint"
    init_line_text2="init"
    for i in range(len(module_lines)):
        if (init_line_text in module_lines[i] and init_line_text2 in module_lines[i]):
            init_start_index = module_lines[i].find("init ")
            temptext = module_lines[i][:init_start_index+len("init ")]
            module_lines[i]=temptext+str(get_init_state_for_model_num(init_states,num_model))+";\n"
            break
    return module_lines        

In [19]:
def add_door_variables_to_init(module_lines,num_doors):
    new_module_lines = [] 
    init_line_text="waypoint"
    init_line_text2="init"
    restart_from = -1
    for i in range(len(module_lines)):
        if (init_line_text in module_lines[i] and init_line_text2 in module_lines[i]):
            restart_from = i; 
            break
        else:
            new_module_lines.append(module_lines[i])
    for i in range(num_doors):
        line_to_add = "door%d:[unknown..open] init unknown;\n" % (i)
        new_module_lines.append(line_to_add)
    for i in range(restart_from,len(module_lines)):
        new_module_lines.append(module_lines[i])
    return new_module_lines
    

In [20]:
def create_new_prism_model(folder,new_folder,filename,new_filename_suffix,prism_model_ext,numStates,
                           states_to_fail_at,num_models,init_states,convert_states_from_labels,door_states):
    #read in the model and extract the module bit
    prism_model_file = open(folder+filename+prism_model_ext)
    prism_model_lines = prism_model_file.readlines()
    prism_model_file.close()
    module_start,module_end,module_lines = get_module_lines(prism_model_lines)
    reward_start,reward_end,reward_lines = get_reward_lines(prism_model_lines)
    labels = get_labels(prism_model_lines)
    
    print labels 
    return 
    #generate states to fail at
    #states_to_fail_at=get_n_states_at_random(num_rand_states,numStates)
    if convert_states_from_labels:
        for i in range(len(states_to_fail_at)):
            states_to_fail_at[i]=labels[states_to_fail_at[i]]
        for i in range(len(init_states)):
            init_states[i]=labels[init_states[i]]
        print "New States To Fail At"
        print states_to_fail_at
        print "New Init States"
        print init_states
            
    #create the new model 
    for i in range(len(states_to_fail_at)):
        state_to_look_at = states_to_fail_at[i]
        new_module_lines=add_failure_to_states(state_to_look_at,module_lines)
        module_lines = new_module_lines
    
    for i in range(len(door_states)):
        door_states_pair = door_states[i]
        print "Adding doors to "+str(door_states_pair)
        state1 = door_states_pair[0]
        state1 = labels[state1]
        state2 = door_states_pair[1]
        state2 = labels[state2]
        
        doornum = i
        (new_module_lines,new_reward_lines) = add_door_to_bothsides(doornum,state1,state2,module_lines,reward_lines)
        #print new_module_lines
        #print new_reward_lines 
        #raw_input()
        module_lines = new_module_lines
        reward_lines = new_reward_lines
    new_module_lines=add_door_variables_to_init(module_lines,len(door_states))
    new_prism_model_lines = create_new_prism_model_lines(new_module_lines,module_start,module_end,new_reward_lines,reward_start,reward_end,prism_model_lines)
    
    for i in range(num_models):
        new_prism_model_lines = modify_init_line(new_prism_model_lines,i,init_states)
        #write to file
        new_filename = create_model_name(new_folder+filename,new_filename_suffix+str(i),prism_model_ext)
        write_model_to_file(new_filename,new_prism_model_lines)

In [32]:
def test(folder,new_folder,filename,new_filename_suffix,prism_model_ext,numStates,
                           states_to_fail_at,num_models,init_states,convert_states_from_labels,door_states):
    #read in the model and extract the module bit
    prism_model_file = open(folder+filename+prism_model_ext)
    prism_model_lines = prism_model_file.readlines()
    prism_model_file.close()
    module_start,module_end,module_lines = get_module_lines(prism_model_lines)
    reward_start,reward_end,reward_lines = get_reward_lines(prism_model_lines)
    labels = get_labels(prism_model_lines)
    
    print module_lines
    
    print labels 
    return 

In [33]:
folder = "../decomp_tests/"
new_tests_folder = "../autogen_testfiles/"
filename = "for_autogen"#"grid_2_topomap"#"grid_map_topmap"
prism_model_ext = ".prism"
#we know that there are 0 to 28 states 
numStates = 33 
num_models = 4
init_states=[19,3,49,30]#[57,50,34,43]
states_to_fail_at=[8,22,27,28,36,47,59]#[63,59,64,61,62,60]
#door_states=[(22,24),(29,26),(4,53),(57,60)]
door_states=[(22,24),(57,60)]
filename_suffix = "sim_doors"
convert_states_from_labels = True
full_folder = new_tests_folder + "/"+filename+"/m"+str(num_models)+"/f"+str(len(states_to_fail_at))+"/d"+str(len(door_states))+"/"
#create_dir(full_folder)

In [35]:
test(folder,full_folder,filename,filename_suffix
                       ,prism_model_ext,numStates,states_to_fail_at,
                       num_models,init_states,convert_states_from_labels,door_states)



['\n', '       s:[failstate..maxstate] init 0;\n', '       \n', '       //the last state is a fail state\n', "       [v0_2] (s=0) -> (s'=2);\n", "       [v2_0] (s=2) -> (s'=0);\n", '       \n', "       [v2_3] (s=2) -> (s'=3);\n", "       [v3_2] (s=3) -> (s'=2);\n", '\n', "       [v2_5] (s=2) -> p:(s'=5);\n", "       [v5_2] (s=5) -> (s'=2);\n", '       \n', "       [v3_4_7] (s=3) -> (s'=4);\n", "       [v4_3_7] (s=4) -> (s'=3);\n", '\n', "       [v3_7] (s=3) -> (s'=7);\n", "       [v7_8] (s=7) -> (s'=8);\n", "       [v8_7] (s=8) -> (s'=7);\n", '       \n', "       [v4_6] (s=4) -> p:(s'=6);\n", "       [v6_4] (s=6) -> (s'=4);\n", '\n', "       [v1_4] (s=1) -> (s'=4);\n", "       [v4_1] (s=4) -> (s'=1);\n", '\n', '       \n', "       [failed] (s=failstate) -> (s'=failstate);\n", '\n']
{}


In [34]:
#create_new_prism_model(folder,full_folder,filename,filename_suffix
#                       ,prism_model_ext,numStates,states_to_fail_at,
#                       num_models,init_states,convert_states_from_labels,door_states)